# Evaluation

In [1]:
utils_path = "../utils/"
corpus_path = "../datasets/cantemist_v6/"
sub_task_path = "cantemist-norm/"
test_gs_path = corpus_path + "test-set/" + sub_task_path

In [2]:
import tensorflow as tf

# Auxiliary components
import sys
sys.path.insert(0, utils_path)
from nlp_utils import *

RES_DIR = "../results/Cantemist/final_exec/"

# GS data
df_test_gs = format_ner_gs(test_gs_path, subtask='norm')

2022-09-14 08:51:26.212789: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


## Evaluation

In [4]:
def check_ner_norm_performance(model_name, arr_execs):
    """
    Sanity-check procedure that prints the NER performance of each single model execution.
    """
    for i_exec in arr_execs:
        print("Exec " + str(i_exec) + ":")
        df_test_preds = pd.read_csv(RES_DIR + "df_test_preds_c_hier_task_iob_" + \
                str(model_name) + "_" + str(i_exec) + ".csv", header=0, sep='\t')
        print("NER performance:", calculate_ner_metrics(
            gs=df_test_gs, 
            pred=format_ner_pred_df(
                gs_path=test_gs_path, 
                df_preds=df_test_preds, 
                subtask='ner'
            ),
            subtask='ner'
        ), end="\n\n")

In [5]:
def model_performance(dict_names_execs, 
                      df_gs=df_test_gs, path_gs=test_gs_path,
                      round_n=3):
    """
    Generate a pd.DataFrame with the statistics of the performance of each model.
    
    dict_names_execs: each key is a string with the model name, and 
                      each value is a list with the execs of the corresponding model.
    """
    res_dict = {}
    for model_name in dict_names_execs:
        p_res, r_res, f1_res = [], [], []
        for i_exec in dict_names_execs[model_name]:
            df_test_preds = pd.read_csv(RES_DIR + "df_test_preds_c_hier_task_iob_" + \
                    str(model_name) + "_" + str(i_exec) + ".csv", header=0, sep='\t')
            p, r, f1 = calculate_ner_metrics(
                gs=df_gs, 
                pred=format_ner_pred_df(
                    gs_path=path_gs, 
                    df_preds=df_test_preds, 
                    subtask='ner'
                ),
                subtask='ner'
            )
            p_res.append(p)
            r_res.append(r)
            f1_res.append(f1)
        p_res_stat = pd.Series(p_res).describe()
        r_res_stat = pd.Series(r_res).describe()
        f1_res_stat = pd.Series(f1_res).describe()
        res_dict[model_name] = {"P_avg": round(p_res_stat['mean'], round_n), "P_std": round(p_res_stat['std'], round_n), 
                                "P_max": round(p_res_stat['max'], round_n),
                                "R_avg": round(r_res_stat['mean'], round_n), "R_std": round(r_res_stat['std'], round_n), 
                                "R_max": round(r_res_stat['max'], round_n),
                                "F1_avg": round(f1_res_stat['mean'], round_n), "F1_std": round(f1_res_stat['std'], round_n), 
                                "F1_max": round(f1_res_stat['max'], round_n)}
    return pd.DataFrame(res_dict, index=["P_avg", "P_std", "P_max", 
                                         "R_avg", "R_std", "R_max", 
                                         "F1_avg", "F1_std", "F1_max"]).transpose()    

In [6]:
def format_df_paper(df_res):
    arr_metrics = ["P", "R", "F1"]
    arr_cols = []
    for metric in arr_metrics:
        df_res[metric + '_avg_std'] = df_res.apply(
            lambda x: "." + str(x[metric + '_avg']).split('.')[-1] + " ± " + \
                "." + str(x[metric + '_std']).split('.')[-1], 
            axis=1
        )
        df_res[metric + '_max'] = df_res[metric + '_max'].apply(
            lambda x: "." + str(x).split('.')[-1]
        )
        arr_cols += [metric + '_avg_std', metric + '_max']
    return df_res[arr_cols]

In [5]:
# Sanity check

In [7]:
m_name = "xlmr"
execs = [1, 2, 3, 4, 5]

check_ner_norm_performance(model_name=m_name, arr_execs=execs)

Exec 1:
NER performance: (0.8627, 0.8632, 0.863)

Exec 2:
NER performance: (0.8587, 0.8662, 0.8624)

Exec 3:
NER performance: (0.8607, 0.8574, 0.8591)

Exec 4:
NER performance: (0.852, 0.8602, 0.856)

Exec 5:
NER performance: (0.8562, 0.8602, 0.8582)



In [8]:
m_name = "xlmr_galen"
execs = [1, 2, 3, 4, 5]

check_ner_norm_performance(model_name=m_name, arr_execs=execs)

Exec 1:
NER performance: (0.8678, 0.8637, 0.8658)

Exec 2:
NER performance: (0.8752, 0.8571, 0.8661)

Exec 3:
NER performance: (0.8733, 0.8726, 0.8729)

Exec 4:
NER performance: (0.8573, 0.8745, 0.8658)

Exec 5:
NER performance: (0.8699, 0.8671, 0.8685)



## Paper

### NER

In [9]:
model_performance(
    {
        'beto': [1, 2, 3, 4, 5], 
        'beto_galen': [1, 2, 3, 4, 5],
        'mbert': [1, 2, 3, 4, 5], 
        'mbert_galen': [1, 2, 3, 4, 5],
        'xlmr': [1, 2, 3, 4, 5], 
        'xlmr_galen': [1, 2, 3, 4, 5]
    }
)

,P_avg,P_std,P_max,R_avg,R_std,R_max,F1_avg,F1_std,F1_max
beto,0.862,0.007,0.871,0.855,0.004,0.860,0.859,0.004,0.864
beto_galen,0.872,0.005,0.879,0.868,0.005,0.872,0.870,0.004,0.875
mbert,0.862,0.011,0.875,0.860,0.010,0.872,0.861,0.003,0.865
mbert_galen,0.871,0.007,0.883,0.868,0.004,0.873,0.870,0.003,0.876
xlmr,0.858,0.004,0.863,0.861,0.003,0.866,0.860,0.003,0.863
xlmr_galen,0.869,0.007,0.875,0.867,0.007,0.874,0.868,0.003,0.873


In [10]:
format_df_paper(
    model_performance(
        {
            'beto': [1, 2, 3, 4, 5], 
            'beto_galen': [1, 2, 3, 4, 5],
            'mbert': [1, 2, 3, 4, 5], 
            'mbert_galen': [1, 2, 3, 4, 5],
            'xlmr': [1, 2, 3, 4, 5], 
            'xlmr_galen': [1, 2, 3, 4, 5]
        }
    )
)

,P_avg_std,P_max,R_avg_std,R_max,F1_avg_std,F1_max
beto,.862 ± .007,.871,.855 ± .004,.86,.859 ± .004,.864
beto_galen,.872 ± .005,.879,.868 ± .005,.872,.87 ± .004,.875
mbert,.862 ± .011,.875,.86 ± .01,.872,.861 ± .003,.865
mbert_galen,.871 ± .007,.883,.868 ± .004,.873,.87 ± .003,.876
xlmr,.858 ± .004,.863,.861 ± .003,.866,.86 ± .003,.863
xlmr_galen,.869 ± .007,.875,.867 ± .007,.874,.868 ± .003,.873


Save the (F1) performance of all executions of all models

In [3]:
def model_f1_values(dict_names_execs, 
                    df_gs=df_test_gs, path_gs=test_gs_path):
    """
    Generate a vector containing the F1 performance of all executions of all models, in the given order.
    
    dict_names_execs: each key is a string with the model name, and 
                      each value is a list with the random execs of the corresponding model.
    """
    arr_values = []
    for model_name in dict_names_execs:
        for i_exec in dict_names_execs[model_name]:
            df_test_preds = pd.read_csv(RES_DIR + "df_test_preds_c_hier_task_iob_" + \
                    str(model_name) + "_" + str(i_exec) + ".csv", header=0, sep='\t')
            _, _, f1 = calculate_ner_metrics(
                gs=df_gs, 
                pred=format_ner_pred_df(
                    gs_path=path_gs, 
                    df_preds=df_test_preds, 
                    subtask='ner'
                ),
                subtask='ner'
            )
            arr_values.append(f1)
    return arr_values

In [ ]:
# NER

In [4]:
arr_val = model_f1_values(
    {
        'beto': [1, 2, 3, 4, 5], 
        'beto_galen': [1, 2, 3, 4, 5],
        'mbert': [1, 2, 3, 4, 5], 
        'mbert_galen': [1, 2, 3, 4, 5],
        'xlmr': [1, 2, 3, 4, 5], 
        'xlmr_galen': [1, 2, 3, 4, 5]
    }
)

In [5]:
pd.DataFrame(arr_val).to_csv(RES_DIR + "ner_f1_exec_c_hier_task.csv", index=False, header=False, sep = '\t')

## Ensemble

In [11]:
# Load texts from test corpus 
test_path = corpus_path + "test-set/" + sub_task_path
test_files = [f for f in os.listdir(test_path) if os.path.isfile(test_path + f) and f.split('.')[-1] == 'txt']
test_data = load_text_files(test_files, test_path)
df_text_test = pd.DataFrame({'doc_id': [s.split('.txt')[0] for s in test_files], 'raw_text': test_data})

# Load test doc list
test_doc_list = sorted(set(df_text_test["doc_id"]))

iob_lab_decoder = {0: "B", 1: "I", 2: "O"}

text_col = "raw_text"

ENS_EVAL_STRAT = 'sum'
RES_DIR_ENS = RES_DIR + "ensemble/"

arr_exec = [1, 2, 3, 4, 5]
model_exec = {
    'mbert': arr_exec,
    'mbert_galen': arr_exec,
    'beto': arr_exec,
    'beto_galen': arr_exec,
    'xlmr': arr_exec,
    'xlmr_galen': arr_exec
}

In [12]:
import pickle

def ens_performance(dict_names_execs, df_gs=df_test_gs, path_gs=test_gs_path, 
                        doc_list=test_doc_list, lab_decoder_list=[iob_lab_decoder],
                        df_text=df_text_test, text_col=text_col, 
                        ens_eval_strategy=ENS_EVAL_STRAT):
    ens_preds, ens_start_end = [], []
    for model_name in dict_names_execs:
        # Word start-send pairs
        with open(RES_DIR_ENS + "test_word_start_end_c_hier_task_iob_" + str(model_name) + ".pck", "rb") as f:
            word_start_end = pickle.load(f)
        for i_exec in dict_names_execs[model_name]:
            # Word predictions
            with open(RES_DIR_ENS + "test_word_preds_c_hier_task_iob_" + str(model_name) + \
                      "_" + str(i_exec) + ".pck", "rb") as f:
                ens_preds.append(pickle.load(f))
                
            ens_start_end.append(word_start_end)
        
    df_ens_ann = ens_ner_preds_brat_format(
        doc_list=doc_list, 
        ens_doc_word_preds=ens_preds,                   
        ens_doc_word_start_end=ens_start_end, 
        lab_decoder_list=lab_decoder_list, 
        df_text=df_text, 
        text_col=text_col, 
        ens_eval_strat=ens_eval_strategy,
        subtask='ner'
    )
    return df_ens_ann, calculate_ner_metrics(
        df_gs,
        pred=format_ner_pred_df(
            gs_path=path_gs, 
            df_preds=df_ens_ann, 
            subtask='ner'
        ),
        subtask='ner'
    )

In [13]:
dic_ens_res = {}

### BETO

In [14]:
arr_model_name = ['beto']

In [15]:
df_ens_ann, res_metrics = ens_performance(dict_names_execs={x: model_exec[x] for x in arr_model_name})
print(res_metrics)

(0.8792, 0.8712, 0.8752)


In [11]:
dic_ens_res['_'.join(arr_model_name)] = res_metrics

In [7]:
df_ens_ann.to_csv(RES_DIR_ENS + "df_test_preds_c_hier_task_iob_" + '_'.join(arr_model_name) + \
                  ".csv", index=False, header=True, sep = '\t')

### BETO-Galén

In [16]:
arr_model_name = ['beto_galen']

In [17]:
df_ens_ann, res_metrics = ens_performance(dict_names_execs={x: model_exec[x] for x in arr_model_name})
print(res_metrics)

(0.8843, 0.8794, 0.8819)


In [15]:
dic_ens_res['_'.join(arr_model_name)] = res_metrics

In [10]:
df_ens_ann.to_csv(RES_DIR_ENS + "df_test_preds_c_hier_task_iob_" + '_'.join(arr_model_name) + \
                  ".csv", index=False, header=True, sep = '\t')

### mBERT

In [18]:
arr_model_name = ['mbert']

In [19]:
df_ens_ann, res_metrics = ens_performance(dict_names_execs={x: model_exec[x] for x in arr_model_name})
print(res_metrics)

(0.8769, 0.8726, 0.8747)


In [18]:
dic_ens_res['_'.join(arr_model_name)] = res_metrics

In [13]:
df_ens_ann.to_csv(RES_DIR_ENS + "df_test_preds_c_hier_task_iob_" + '_'.join(arr_model_name) + \
                  ".csv", index=False, header=True, sep = '\t')

### mBERT-Galén

In [20]:
arr_model_name = ['mbert_galen']

In [21]:
df_ens_ann, res_metrics = ens_performance(dict_names_execs={x: model_exec[x] for x in arr_model_name})
print(res_metrics)

(0.8847, 0.8811, 0.8829)


In [21]:
dic_ens_res['_'.join(arr_model_name)] = res_metrics

In [16]:
df_ens_ann.to_csv(RES_DIR_ENS + "df_test_preds_c_hier_task_iob_" + '_'.join(arr_model_name) + \
                  ".csv", index=False, header=True, sep = '\t')

### XLM-R

In [22]:
arr_model_name = ['xlmr']

In [23]:
df_ens_ann, res_metrics = ens_performance(dict_names_execs={x: model_exec[x] for x in arr_model_name})
print(res_metrics)

(0.874, 0.8723, 0.8731)


In [24]:
dic_ens_res['_'.join(arr_model_name)] = res_metrics

In [19]:
df_ens_ann.to_csv(RES_DIR_ENS + "df_test_preds_c_hier_task_iob_" + '_'.join(arr_model_name) + \
                  ".csv", index=False, header=True, sep = '\t')

### XLM-R-Galén

In [24]:
arr_model_name = ['xlmr_galen']

In [25]:
df_ens_ann, res_metrics = ens_performance(dict_names_execs={x: model_exec[x] for x in arr_model_name})
print(res_metrics)

(0.8841, 0.8794, 0.8817)


In [27]:
dic_ens_res['_'.join(arr_model_name)] = res_metrics

In [22]:
df_ens_ann.to_csv(RES_DIR_ENS + "df_test_preds_c_hier_task_iob_" + '_'.join(arr_model_name) + \
                  ".csv", index=False, header=True, sep = '\t')

### BETO + BETO-Galén

In [26]:
arr_model_name = ['beto', 'beto_galen']

In [27]:
df_ens_ann, res_metrics = ens_performance(dict_names_execs={x: model_exec[x] for x in arr_model_name})
print(res_metrics)

(0.8885, 0.8772, 0.8828)


In [30]:
dic_ens_res['_'.join(arr_model_name)] = res_metrics

In [25]:
df_ens_ann.to_csv(RES_DIR_ENS + "df_test_preds_c_hier_task_iob_" + '_'.join(arr_model_name) + \
                  ".csv", index=False, header=True, sep = '\t')

### mBERT + mBERT-Galén

In [28]:
arr_model_name = ['mbert', 'mbert_galen']

In [29]:
df_ens_ann, res_metrics = ens_performance(dict_names_execs={x: model_exec[x] for x in arr_model_name})
print(res_metrics)

(0.8856, 0.8803, 0.8829)


In [33]:
dic_ens_res['_'.join(arr_model_name)] = res_metrics

In [28]:
df_ens_ann.to_csv(RES_DIR_ENS + "df_test_preds_c_hier_task_iob_" + '_'.join(arr_model_name) + \
                  ".csv", index=False, header=True, sep = '\t')

### XLM-R + XLM-R-Galén

In [30]:
arr_model_name = ['xlmr', 'xlmr_galen']

In [31]:
df_ens_ann, res_metrics = ens_performance(dict_names_execs={x: model_exec[x] for x in arr_model_name})
print(res_metrics)

(0.8855, 0.8816, 0.8836)


In [36]:
dic_ens_res['_'.join(arr_model_name)] = res_metrics

In [31]:
df_ens_ann.to_csv(RES_DIR_ENS + "df_test_preds_c_hier_task_iob_" + '_'.join(arr_model_name) + \
                  ".csv", index=False, header=True, sep = '\t')

### BETO + mBERT + XLM-R

In [32]:
arr_model_name = ['beto', 'mbert', 'xlmr']

In [33]:
df_ens_ann, res_metrics = ens_performance(dict_names_execs={x: model_exec[x] for x in arr_model_name})
print(res_metrics)

(0.8838, 0.8775, 0.8807)


In [39]:
dic_ens_res['_'.join(arr_model_name)] = res_metrics

In [34]:
df_ens_ann.to_csv(RES_DIR_ENS + "df_test_preds_c_hier_task_iob_" + '_'.join(arr_model_name) + \
                  ".csv", index=False, header=True, sep = '\t')

### BETO-Galén + mBERT-Galén + XLM-R-Galén

In [34]:
arr_model_name = ['beto_galen', 'mbert_galen', 'xlmr_galen']

In [35]:
df_ens_ann, res_metrics = ens_performance(dict_names_execs={x: model_exec[x] for x in arr_model_name})
print(res_metrics)

(0.8917, 0.8863, 0.889)


In [42]:
dic_ens_res['_'.join(arr_model_name)] = res_metrics

In [37]:
df_ens_ann.to_csv(RES_DIR_ENS + "df_test_preds_c_hier_task_iob_" + '_'.join(arr_model_name) + \
                  ".csv", index=False, header=True, sep = '\t')